In [198]:
import requests as r
from bs4 import BeautifulSoup
def scrape_page_range(query, start, end):
    items = []
    for i in range(start, end+1):
        req = r.get("https://www.ebay-kleinanzeigen.de/s-pc-zubehoer-software/seite:{}/{}/k0c225".format(i,query))
        html_doc = req.text
        items.extend(get_items_on_page(html_doc))
    return items

In [199]:
def get_items_on_page(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    items_ul = soup.select_one("#srchrslt-adtable")
    all_items = items_ul.find_all("li")
    items_list = []
    for item in all_items:
        try:
            section1 = item.article.select_one("section.aditem-main")
            section2 = item.article.select_one("section.aditem-details")
            title = section1.h2.a.text
            url = section1.h2.a.attrs['href']
            short_desc = section1.p.text
            price = section2.strong.text
            address = ", ".join(section2.findChildren()[1].get_text().split())
            items_list.append([title, price, short_desc, address, url])
        except AttributeError:
            pass
    return items_list

In [200]:
items = scrape_page_range('gtx-780', 1,6)

In [201]:
import pandas as pd
import re
df = pd.DataFrame(items, columns=['title', 'price', 'desc', 'address', 'url'])
df.price = df.price.apply(lambda x: re.sub(r"\D", "", x))
df.price = pd.to_numeric(df.price)
df = df.sort_values("price")

In [202]:
df.shape

(69, 5)

In [203]:
df.head(15)

,title,price,desc,address,url
49,Kühler EVGA GeForce GTX 780 unbenutzt,20.0,"Hallo,\n\nVerkaufe unbenutzten Kühler(nur der ...","07745, Jena",/s-anzeige/kuehler-evga-geforce-gtx-780-unbenu...
66,GTX 780 Wasserkühlung,30.0,EK-Wasserkühler für ASUS R9 290(X) DirectCU II...,"74653, Künzelsau",/s-anzeige/gtx-780-wasserkuehlung/538279962-22...
0,Original Gainward Phenom GTX 780 Ti Kühlkörper...,35.0,Verkaufe meinen original Lüfter der Gainward G...,"70567, Möhringen",/s-anzeige/original-gainward-phenom-gtx-780-ti...
20,EKWB GTX Titan/GTX 780 - Acetal+Nickel Kühlblo...,35.0,Ich verkaufe meinen Kühlblock für eine GTX 780...,"52074, Laurensberg",/s-anzeige/ekwb-gtx-titan-gtx-780-acetal-nicke...
23,Inno3D GeForce GTX 780 iChill HerculeZ X3 Ultr...,60.0,Hallo zusammen\n\nverkaufe hier eine defekte I...,"71106, Magstadt",/s-anzeige/inno3d-geforce-gtx-780-ichill-hercu...
11,Gigabyte GTX 780 Ghz Edition in OVP,100.0,Gigabyte GTX 780 Ghz Edition\n\n- Top Zustand ...,"04157, Nord",/s-anzeige/gigabyte-gtx-780-ghz-edition-in-ovp...
2,ASUS GeForce GTX 780 Direct CU II OC mit 3GB V...,105.0,"Hallo,\n\nich verkaufe meine ASUS GeForce GTX ...","48527, Nordhorn",/s-anzeige/asus-geforce-gtx-780-direct-cu-ii-o...
13,ASUS GeForce GTX 780 Direct CU II OC mit 3GB V...,105.0,"Hallo,\n\nich verkaufe meine ASUS GeForce GTX ...","48527, Nordhorn",/s-anzeige/asus-geforce-gtx-780-direct-cu-ii-o...
6,ZOTAC GeForce GTX 780 AMP!,115.0,Verkaufe hier meine geliebte GTX 780 in gutem ...,"35745, Herborn",/s-anzeige/zotac-geforce-gtx-780-amp-/62606056...
1,Gtx 780 Windforce x3 Edition,115.0,Ohne Ovp funktioniert jedoch einwandfrei.\nVer...,"04105, Mitte",/s-anzeige/gtx-780-windforce-x3-edition/626586...


In [160]:
df.ix[5].url

'/s-anzeige/powercolor-r9-280x-turbo-duo-3gb-siehe-video-beweis-/626370312-225-3005'